<a href="https://colab.research.google.com/github/newmantic/Dinic/blob/main/Dinic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import deque, defaultdict

class Dinic:
    def __init__(self, graph, source, sink):
        """
        Initialize the Dinic's algorithm.
        :param graph: A dictionary where the keys are nodes and the values are dictionaries
                      of neighbors with the capacity of the edge as the value.
                      For example: {0: {1: 16, 2: 13}, 1: {2: 10, 3: 12}, ...}
        :param source: The source node.
        :param sink: The sink node.
        """
        self.graph = graph
        self.residual_graph = defaultdict(dict)
        self.source = source
        self.sink = sink
        self.level = {}

        for u in graph:
            for v in graph[u]:
                self.residual_graph[u][v] = graph[u][v]
                if v not in self.residual_graph:
                    self.residual_graph[v] = {}
                if u not in self.residual_graph[v]:
                    self.residual_graph[v][u] = 0  # Add reverse edge with 0 capacity

    def bfs(self):
        """
        Perform BFS to construct the level graph.
        :return: True if there is a path from source to sink, False otherwise.
        """
        self.level = {u: -1 for u in self.residual_graph}
        self.level[self.source] = 0
        queue = deque([self.source])

        while queue:
            u = queue.popleft()

            for v in self.residual_graph[u]:
                if self.level[v] < 0 and self.residual_graph[u][v] > 0:  # Edge has remaining capacity
                    self.level[v] = self.level[u] + 1
                    queue.append(v)

        return self.level[self.sink] >= 0

    def dfs(self, u, flow):
        """
        Perform DFS to send flow along the path found in the level graph.
        :param u: The current node.
        :param flow: The flow to send.
        :return: The flow sent along the path.
        """
        if u == self.sink:
            return flow

        total_flow_sent = 0
        for v in list(self.residual_graph[u]):
            if self.level[v] == self.level[u] + 1 and self.residual_graph[u][v] > 0:
                current_flow = min(flow, self.residual_graph[u][v])
                temp_flow = self.dfs(v, current_flow)

                if temp_flow > 0:
                    self.residual_graph[u][v] -= temp_flow
                    self.residual_graph[v][u] += temp_flow
                    total_flow_sent += temp_flow
                    flow -= temp_flow

                    if flow == 0:
                        break

        return total_flow_sent

    def dinic(self):
        """
        Execute Dinic's algorithm to find the maximum flow from source to sink.
        :return: The maximum flow value.
        """
        max_flow = 0

        while self.bfs():
            flow = self.dfs(self.source, float('Inf'))
            while flow:
                max_flow += flow
                flow = self.dfs(self.source, float('Inf'))

        return max_flow

In [2]:
def test_case_1():
    graph = {
        0: {1: 16, 2: 13},
        1: {2: 10, 3: 12},
        2: {1: 4, 4: 14},
        3: {2: 9, 5: 20},
        4: {3: 7, 5: 4},
        5: {}
    }
    dinic = Dinic(graph, 0, 5)
    max_flow = dinic.dinic()
    print(f"Max Flow: {max_flow}")  # Expected: 23

test_case_1()

Max Flow: 23


In [3]:
def test_case_2():
    graph = {
        0: {1: 10, 2: 10},
        1: {3: 4, 2: 2, 4: 8},
        2: {4: 9},
        3: {5: 10},
        4: {3: 6, 5: 10},
        5: {}
    }
    dinic = Dinic(graph, 0, 5)
    max_flow = dinic.dinic()
    print(f"Max Flow: {max_flow}")  # Expected: 19

test_case_2()

Max Flow: 19


In [4]:
def test_case_3():
    graph = {
        0: {1: 20, 2: 10},
        1: {2: 5, 3: 10},
        2: {4: 10},
        3: {5: 20},
        4: {3: 15, 5: 10},
        5: {}
    }
    dinic = Dinic(graph, 0, 5)
    max_flow = dinic.dinic()
    print(f"Max Flow: {max_flow}")  # Expected: 20

test_case_3()

Max Flow: 20
